## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_pdf import PdfPages
import torch
import sys
import os
from torch.utils.data import DataLoader, random_split
import wandb
import random
import datetime

current_dir = os.path.dirname(os.path.abspath(''))
two_up_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(two_up_dir)

from TinyCenterSpeed.src.models.resnet import *
from TinyCenterSpeed.src.models.CenterSpeed import *
from TinyCenterSpeed.dataset.CenterSpeed_dataset import *
from TinyCenterSpeed.src.models.losses import *
from train import *


%env "WANDB_NOTEBOOK_NAME" "centerspeed.ipynb"
print(wandb.__version__)
wandb.login()

## Load Data

In [ ]:
def cartesian_to_pixel(x, y, image_size=[64,64], pixel_size=0.1):
    pixel_x = int(x / pixel_size + image_size[0] / 2)
    pixel_y = int(y / pixel_size + image_size[1] / 2)
    return pixel_x, pixel_y

# 데이터 증강 설정 (45도 회전 및 50% 확률로 좌우 반전) 근데 지금 None으로 설정 되어있음 
transform = transforms.Compose([RandomRotation(45),
                                RandomFlip(0.5)])

#set = CenterSpeedDataset('../../dataset/data/CenterSpeedDataset', transform=None, dense=True)
# 이 경로에 있는 여러 파일을 자동으로 로드하도록 만들어짐
set = CenterSpeedDataset('/home/harry/ros2_ws/src/TinyCenterSpeed/dataset/data/TinyCenterSpeed_dataset/data/CenterSpeedDataset', transform=None, dense=True)
# 이 dataset 함수에서 반환하는 값이 input, gt_heatmap, data, dense_data, is_free 이렇게 5개임

set.seq_len = 2 # 2개의 프레임 입력
# 학습 시 스케일링 파라미터 설정 이게 heatmap 만들때 스케일 된 좌표를 사용하는건데 센서 데이터의 오차가 생길 수 있음을 고려해서 만든거지 
# 1.0이면 원본 크기, 0.9면 10% 축소
set.sx = 0.9
set.sy = 0.9
set.change_image_size(64)
set.change_pixel_size(0.1)

# Fixed: Use absolute path for validation_set.csv
# validation_set.csv 파일의 절대 경로를 사용하여 데이터셋을 로드
independent_test_set = LidarDatasetSeqOD('/home/harry/ros2_ws/src/TinyCenterSpeed/dataset/data/TinyCenterSpeed_dataset/data/validation_set.csv')
independent_test_set.seq_len = 2

# 

In [ ]:
input, gt_heatmap, data, dense_data, is_free = set[1]  
# input은 (3, 64, 64) 형태로 되어있음
# gt_heatmap은 (64, 64) 형태로 되어있음
# data는 (x, y, yaw, vx, vy, yaw_rate) 형태로 되어있음
# dense_data는 (64, 64, 3) 형태로 되어있음
# is_free는 (1, 64, 64) 형태로 되어있음

print(f'Data: GT')
# plt.imshow(input[1].numpy(), cmap='gray')
plt.imshow(gt_heatmap)
plt.colorbar() #색상 스케일 바
plt.show()

labels = ['VX', 'VY', 'YAW']
print(f'Dense data shape: {dense_data.shape}\n')

for i in range(dense_data.shape[2]):
    plt.imshow(dense_data[:,:,i])
    print(f'Data: {labels[i]}')
    print(f'Maximum from data: {data[i+2].item()}')
    # Extract and print the maximum value
    max_value = np.max(dense_data[:,:,i].numpy().flatten())
    min_value = np.min(dense_data[:,:,i].numpy().flatten())
    print(f"Maximum value in image slice {i}: {max_value}")
    print(f"Minimum value in image slice {i}: {min_value}")
    plt.colorbar()
    plt.show()



In [ ]:
### TEST THE MODEL

model = CenterSpeedDense() # 모델 초기화, 가중치는 랜덤
model.eval() #평가 모드(드롭아웃(학습 할 때 뉴런 일부를 무작윙로 꺼버리는 법), 배치 정규화 비활성화)

output = model(input.unsqueeze(0)) #(6, 64, 64) → (1, 6, 64, 64) 배치 차원 추가
#pytorch 모델의 입력은 항상 아래 형태를 기대함 
# B, C, H, W = 배치 크기. 체널수, 높이, 너비

plt.imshow(input[0])
plt.show()
for i in range(output.shape[1]): #output.shape[1]은 4임 (채널 수)
    plt.imshow(output[0,i].detach().numpy()) #detach : 그래디언트 분리
    plt.colorbar()
    plt.show()

In [ ]:
### new Loss funcction

def dense_loss(output, gt_heatmap, gt_dense_data, is_free, alpha=0.7, decay=1):
    print(f'Output Shape: {output.shape}')
    print(f'GT Heatmap Shape: {gt_heatmap.shape}')
    print(f'GT Dense Shape: {gt_dense_data.shape}')
    # fig, ax = plt.subplots(1,3, figsize=(15,5))
    # ax[0].imshow(gt_heatmap[0])
    # ax[0].set_title('GT Heatmap')
    # ax[1].imshow(gt_dense_data[:,:,:,0].squeeze())
    # ax[1].set_title('GT Dense Data')
    # ax[2].imshow(output[0,:,:,0].squeeze().detach().numpy())


    loss = 0
    batch_size = output.shape[0]

    w = gt_heatmap # unit heatmap 정답 히트맵을 그대로 가중치로 사용 

    #output은 (1, 64, 64, 4) 형태로 되어있음
    # 64*64 픽셀 모두에 대해 loss 계산
    loss += (alpha * (1+w)* (output[:,:,:,0].unsqueeze(-1) - gt_heatmap)**2).sum() #위치  
    loss += ((1-alpha) * (1+w)* (output[:,:,:,1:] - gt_dense_data)**2).sum() #vx, vy, yaw

    return loss/ batch_size

# print(f'Output Shape: {output.shape}')
plt.imshow(output[0,0].detach().numpy())
plt.show()
loss = dense_loss(output.permute(0,2,3,1), gt_heatmap.unsqueeze(0).unsqueeze(-1), dense_data.unsqueeze(0), is_free)
# print(f'Loss: {loss.item()}')

## Parameter Definition

In [ ]:
use_wandb = True  # WandB 활성화
save_code = True
# Define the hyperparameters, logged in wandb

#backbone = 이미지 feature 추출 
#heatmap head = 객체의 중심점 예측하는 출력
#Dense head = 중심점 외의 정보 예측 

epochs = 15
learning_rate = 5e-4
learning_rate_hm = 0.005
learning_rate_head = 0.005
architecture = "CenterSpeed: Hourglass Deep with Sigmoid, & Dropout, BatchNorm and Head with 2 frames, lower resolution: 64x64, pixelsize 0.1"
dataset = "Transfer learning test"
optimizer = "Adam"
batch_size = 32
timer = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = "CenterSpeed_" + timer
loss_used = "CenterSpeedLossFreev2 with updated logic for free tracks on the dataset level"

#wandb configurations
config = {
    "epochs": epochs,
    "learning_rate": learning_rate,
    "architecture": architecture,
    "dataset": dataset,
    "optimizer": optimizer,
    "Loss-Function": loss_used
}
if use_wandb:
    #initialize wandb run
    run = wandb.init(project="CenterSpeedLowRes", config=config, name=run_name, save_code=save_code)#initialize wandb


# 데이터 셋 나누는 부분
# val은 학습 중간 중간 매 epoch 마다 평가
# test는 학습이 끝난 후 평가
train_size = int(len(set) * 1)  # 전체 데이터 셋 중 100%를 훈련에 사용
val_size = int(len(set) * 0)  # 0% for validation since this set is seperate!
test_size = len(set) - (train_size + val_size)  # Remaining 5% for testing

train_dataset, val_dataset, test_dataset = random_split(set, [train_size, val_size, test_size])


training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testing_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
validation_loader = DataLoader(independent_test_set, batch_size=batch_size, shuffle= False)

print("Size of Training Set: ", len(train_dataset))
print("Size of Testing Set: ", len(test_dataset))
print("Size of Validation Set: ", len(val_dataset))

net = CenterSpeedDense(image_size=64)

optimizer = torch.optim.Adam(net.parameters(), lr= learning_rate)
print("Optimizer Initialized")

loss_fn = dense_loss
print("Loss function initialized")

In [ ]:
def print_learning_rates(optimizer):
    for i, param_group in enumerate(optimizer.param_groups):
        print(f"Learning rate of layer {i}: {param_group['lr']}")

print_learning_rates(optimizer)

net.train()

for name, param in net.named_parameters():
    if param.requires_grad:
        print("GRAD: ", name)
    else:
        print("NO GRAD: ", name)

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# from IPython.display import clear_output, display


# def train_epoch_Centerspeed_dense(training_loader, net, optimizer, loss_fn, device = 'cpu', use_wandb=False, pdf=None):
#     running_loss = 0.
#     last_loss = 0.

#     plt.ion()
#     fig, ax = plt.subplots(1,3, figsize=(15,5))

#     for i, data in enumerate(training_loader):
#         # Every data instance is an input + label pair
#         inputs, gts, data, dense_data, is_free = data
#         inputs = inputs.to(device)
#         gts = gts.to(device)
#         data = data.to(device)
#         # Zero your gradients for every batch!
#         optimizer.zero_grad()

#         # Make predictions for this batch
#         output = net(inputs)
#         # Compute the loss and its gradients

#         loss = loss_fn(output.permute(0,2,3,1), gts.unsqueeze(-1), dense_data, is_free)
#         loss.backward()

#         # Adjust learning weights
#         optimizer.step()

#         # Gather data and report
#         running_loss += loss.item()

#         last_loss = loss.item() # loss per batch

#         ##Plot the input, output and ground truth in a interactive plot
#         for a in ax:
#             a.clear()
#         ax[0].imshow(inputs[0,0])
#         ax[0].set_title('Input')
#         ax[1].imshow(output[0,0].detach().numpy())
#         ax[1].set_title('Output')
#         ax[2].imshow(gts[0])
#         ax[2].set_title('Ground Truth')
#         print(np.max(output[0,0].detach().numpy()))
#         clear_output(wait=True)
#         display(fig)


#         print('  batch {} loss: {}'.format(i + 1, last_loss))
#         if use_wandb:
#             wandb.log({"batch_loss": last_loss/len(inputs)})#log the average loss per batch

#     plt.show()
#     return last_loss


In [ ]:
from IPython.display import clear_output, display

def train_epoch_Centerspeed_dense(training_loader, net, optimizer, loss_fn, device = 'cpu', use_wandb=True, pdf=None):
    running_loss = 0.
    last_loss = 0.

    plt.ion()
    fig, ax = plt.subplots(1,3, figsize=(15,5))

    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, gts, data, dense_data, is_free = data
        inputs = inputs.to(device)
        gts = gts.to(device)
        data = data.to(device)
        dense_data = dense_data.to(device)  # 이 줄 추가!
        is_free = is_free.to(device)        # 이 줄도 추가!
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        output = net(inputs)
        # Compute the loss and its gradients

        loss = loss_fn(output.permute(0,2,3,1), gts.unsqueeze(-1), dense_data, is_free)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

        last_loss = loss.item() # loss per batch

        ##Plot the input, output and ground truth in a interactive plot
        for a in ax:
            a.clear()
        ax[0].imshow(inputs[0,0].cpu())  # CPU로 이동해서 시각화
        ax[0].set_title('Input')
        ax[1].imshow(output[0,0].detach().cpu().numpy())  # CPU로 이동
        ax[1].set_title('Output')
        ax[2].imshow(gts[0].cpu())  # CPU로 이동
        ax[2].set_title('Ground Truth')
        print(np.max(output[0,0].detach().cpu().numpy()))
        clear_output(wait=True)
        display(fig)

        print('  batch {} loss: {}'.format(i + 1, last_loss))
        if use_wandb:
            wandb.log({"batch_loss": last_loss/len(inputs)})#log the average loss per batch

    plt.show()
    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
#net, optimizer, loss_fn, training_loader, validation_loader = initialize(config)
net.to(device)

# EPOCHS = 10000
EPOCHS = 10
losses = []

for epoch in range(EPOCHS):
    print('Epoch: ', epoch)
    net.train()
    avg_loss = train_epoch_Centerspeed_dense(training_loader=training_loader, net=net, optimizer=optimizer,loss_fn=loss_fn, device=device, use_wandb=True)

    losses.append(avg_loss)

    # save the model every 5 epochs instead of 1000
    # if epoch % 10 == 0:
    if epoch % 5 == 0:
        model_path = '/home/harry/ros2_ws/src/TinyCenterSpeed/src/trained_models/' + str(epoch) + '.pt'
        torch.save(net.state_dict(), model_path)
        if use_wandb:
            wandb.save(model_path)
        print(f"Model saved at epoch {epoch}")

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
for name, param in net.named_parameters():
    if param.grad is None:
        print(name, param.data)
    if param.grad is not None:
        print(name)



In [ ]:
input = next(iter(training_loader))[0]

import time
start = time.perf_counter()
output = net(input)
end = time.perf_counter()
print(f'Elapsed time: {(end-start)*1000} ms')
print(f'Input Shape: {input.shape}')
print(f'Output Shape: {output.shape}')

input, gt_heatmap, data, dense_data, is_free = set[100]
output = net(input.unsqueeze(0))

In [ ]:

print(output.shape)

In [ ]:
input, gt_heatmap, data, dense_data, is_free = set[np.random.randint(0, len(set))]
input = input.unsqueeze(0)

plt.imshow(input[0,0])
plt.show()
max_index = np.unravel_index(output[0,0].detach().numpy().argmax(), output[0,0].shape)
output[0,0] = F.softmax(output[0,0])
print(data)
for i in range(output.shape[1]):
    plt.imshow(output[0,i].detach().cpu().numpy())
    plt.plot(max_index[1], max_index[0], 'ro')
    plt.colorbar()
    plt.show()
    if i == 0:
        print(f'Output: Heatmap')
        continue

    value_at_max = output[0,i].detach().cpu().numpy()[max_index]
    max_value = np.max(output[0,i].detach().cpu().numpy())
    min_value = np.min(output[0,i].detach().cpu().numpy())

    print(f'GT: {data[1+i]} ')
    print(f'Output: {value_at_max} ')
    print(f'Maximum value in image slice {i}: {max_value}')
    print(f'Minimum value in image slice {i}: {min_value}')
